In [1]:
import os

In [2]:
HUGGINGFACEHUB_API_TOKEN = input("Enter HuggingFace token")

In [20]:
#for huggingface inference api
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [4]:
from langchain.chains import LLMChain
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator

#### Load and split document and create a vector store with word embeddings

In [5]:
#load embedding model
from langchain.embeddings import HuggingFaceEmbeddings

import torch
#Use GPU if available
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

model_name = "intfloat/e5-large-v2"
model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [78]:
#split document into chunks of text
from PyPDF2 import PdfReader

reader = PdfReader("thinkpad_t14_gen_2.pdf")

print(reader.pages[8].extract_text())

Microphone*
 IR camera* / Camera*
Webcam privacy shutter*
 Touch screen*
Power button
 TrackPoint pointing stick
Fingerprint reader*
 Trackpad
NFC label*
 TrackPoint buttons
Speaker
* for selected models
Webcam privacy shutter
Slide the webcam privacy shutter to cover or uncover the camera lens. It is designed to protect your privacy.
Power button
Press to turn on the computer or put the computer to sleep mode.
To turn off the computer, open the Start menu, click 
 Power, and then select Shut down.
The indicator in the power button shows the system status of your computer. 
• Blinking for three times: The computer is initially connected to power.
• On: The computer is on.
• Off: The computer is off or in hibernation mode.
• Blinking rapidly (for Intel models): The computer is entering sleep or hibernation mode.
• Blinking rapidly (for AMD models): The computer is entering hibernation mode.
• Blinking slowly: The computer is in sleep mode.
Related topics
• “Log in with your face ID (for

In [7]:
import re

page_delimiter = '\n'
docu = ""
#exclude cover page and table of contexts
for page in reader.pages[6:]:
    page_text = page.extract_text()
    #remove page number and chapter at bottom of page
    page_text = re.sub(r"\n.*$", "", page_text)
    page_text += page_delimiter
    docu += page_text

In [8]:
print(docu[:1000])

Chapter 1.Meet your computer
ThinkPad T14 Gen 2 and ThinkPad P14s Gen 2 might be shipped with different microprocessors, namely 
Intel® and Advanced Micro Devices (AMD). ThinkPad T15 Gen 2 and ThinkPad P15s Gen 2 are shipped with 
Intel microprocessors. Hereby Intel models and AMD models are used to distinguish computer models with 
different microprocessors.
Front view
T14 Gen 2 and P14s Gen 2
Microphone*
 Infrared (IR) camera* / Camera*
Webcam privacy shutter*
 Touch screen*
Power button
 TrackPoint® pointing stick
RFID label1
 Fingerprint reader*
Trackpad
 Near field communication (NFC) label*
TrackPoint buttons
 Speaker
* for selected models
1 for selected Intel models
T15 Gen 2 and P15s Gen 2
Microphone*
 IR camera* / Camera*
Webcam privacy shutter*
 Touch screen*
Power button
 TrackPoint pointing stick
Fingerprint reader*
 Trackpad
NFC label*
 TrackPoint buttons
Speaker
* for selected models
Webcam privacy shutter
Slide the webcam privacy shutter to cover or uncover the camera le

In [9]:
#split document using recursive splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1500,
    chunk_overlap  = 500,
    length_function = len,
)

texts = text_splitter.split_text(docu)

In [10]:
print(len(texts))

79


In [11]:
print(texts[10])

• Some gestures are not available if the last action was done from the TrackPoint pointing device.
• Some gestures are only available when you are using certain apps.
• If the trackpad surface is stained with oil, turn off the computer first. Then, gently wipe the trackpad 
surface with a soft and lint-free cloth moistened with lukewarm water or computer cleaner.
For more gestures, see the help information of the pointing device.
Disable the trackpad
The trackpad is active by default. To disable the device: 
1. Open the Start menu, and then click Settings ➙ Devices ➙ Touchpad.
2. In the Touchpad section, turn off the Touchpad control.
Use the touch screen (for selected models)
If your computer display supports the multi-touch function, you can navigate the screen with simple touch 
gestures.
Note: Some gestures might not be available when you are using certain apps.
Tap once to single click Tap twice quickly to double-click
Tap and hold to right-click Slide to scroll through items
Zoom

In [12]:
print(texts[11])

Tap and hold to right-click Slide to scroll through items
Zoom out Zoom in
Swipe from the left: view all open windows Swipe from the right: open action center
Swipe downwards shortly: show title bar
Swipe downwards: close the current appDrag
Maintenance tips:
• Turn off the computer before cleaning the touch screen.
• Use a dry, soft, and lint-free cloth or a piece of absorbent cotton to remove fingerprints or dust from the 
touch screen. Do not apply solvents to the cloth.
• The touch screen is a glass panel covered with a plastic film. Do not apply pressure or place any metallic 
object on the screen, which might damage the touch panel or cause it to malfunction.
• Do not use fingernails, gloved fingers, or inanimate objects for input on the screen.
• Regularly calibrate the accuracy of the finger input to avoid a discrepancy.
Connect to an external display
Connect your computer to a projector or a monitor to give presentations or expand your workspace.
Connect to a wired display
If 

In [13]:
#create embedding database using Chroma
docsearch = Chroma.from_texts(texts, embeddings, persist_directory="thinkpad_chromadb")
docsearch.persist()

In [13]:
#lood db if it has already been saved locally
docsearch = Chroma(persist_directory="thinkpad_chromadb", embedding_function=embeddings)

#### Load LLM model

##### Using local instance of HuggingFace model

In [6]:
#for HuggingFace
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, PreTrainedModel, AutoConfig
from langchain.llms import HuggingFacePipeline

In [7]:
"""model_id = "meta-llama/Llama-2-7b-chat-hf
"""

In [12]:
"""#Load HuggingFace model and tokenizer locally
#Initialize tokenizer and model

print("loading model...", end='', flush=True)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
config = AutoConfig.from_pretrained(model_id, max_length=3000, trust_remote_code=True)
config.init_device = 'cuda:0' # For fast initialization directly on GPU!
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, config=config)
print("completed")"""

'#Load model and tokenizer locally\n#Initialize tokenizer and model\nmodel_id="tiiuae/falcon-7b-instruct"\n\nprint("loading model...", end=\'\', flush=True)\ntokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)\nconfig = AutoConfig.from_pretrained(model_id, max_length=3000, trust_remote_code=True)\nconfig.init_device = \'cuda:0\' # For fast initialization directly on GPU!\nmodel = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, config=config)\nprint("completed")'

In [13]:
"""#Load HuggingFace Pipeline
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    config={"temperature":0.2, "max_length":3000, "max_new_tokens":200, "max_time":None , "num_return_sequences":1, "repetition_penalty": 10}, 
    device='cuda:0'
)

llm = HuggingFacePipeline(
    pipeline=pipe,
)"""

'pipe = pipeline(\n    "text-generation", \n    model=model, \n    tokenizer=tokenizer, \n    config={"temperature":0.2, "max_length":3000, "max_new_tokens":200, "max_time":None , "num_return_sequences":1, "repetition_penalty": 10}, \n    device=\'cuda:0\'\n)\n\nllm = HuggingFacePipeline(\n    pipeline=pipe,\n)'

In [13]:
"""#Load HuggingFace model locally using a pipeline

llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",#
    pipeline_kwargs={"temperature":0.2, "max_new_tokens":500, "max_time":None , "num_return_sequences":1},
)
"""

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.1+cu118 with CUDA 1108 (you have 2.0.1+cpu)
    Python  3.10.11 (you have 3.10.10)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


##### Using HuggingFace Inference API

In [45]:
#load model from HuggingFaceHub
from langchain import HuggingFaceHub

repo_id = "tiiuae/falcon-7b-instruct"

llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":0.2,"max_new_tokens":500, "max_time":None , "num_return_sequences":1, "repetition_penalty":10})

### Retrieval QA Chain

In [22]:
query = "How to disable fast startup on my thinkpad?"
#search for the most relevant chunk based on cosine similarity(for Chroma) or Euclidean distance(for FAISS) - results are the same
#k is the number of documents to retrieve
docs = docsearch.similarity_search(query, k=1)

In [23]:
#most similar document
print(docs[0].page_content)

• Always On USB 3.2 connector Gen 1 board
• Always On USB 3.2 connector Gen 1 board bracket
• Coin-cell battery
• Keyboard
• Memory module*
• Speaker assembly
• Wireless WAN card*
* for selected models
Note: Replacement of any parts not listed above, including the built-in rechargeable battery, must be done 
by a Lenovo-authorized repair facility or technician. Go to https://support.lenovo.com/partnerlocation for more 
information.
Disable Fast Startup and the built-in battery
Before replacing any CRU, ensure that you disable Fast Startup first and then disable the built-in battery.
To disable Fast Startup: 
1. Go to Control Panel and view by Large icons or Small icons.
2. Click Power Options, and then click Choose what the power buttons do on the left pane.
3. Click Change settings that are currently unavailable at the top.
4. If prompted by User Account Control (UAC), click Yes.
5. Clear the Turn on fast startup check box, and then click Save changes.
To disable the built-in battery:

In [24]:
#create custom prompt
from langchain.prompts import PromptTemplate

question_prompt_template = """Use the context provided to answer the question. If the context provided does not contain sufficient information to answer the question, say "Sorry, I do not have enough knowledge to answer your question.".

Context:
{context}

Question:
{question}

Answer:
"""

QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

In [25]:
qa = load_qa_chain(llm=llm, chain_type="stuff", prompt=QUESTION_PROMPT)

In [26]:
output = qa({"input_documents": docs, "question": query}, return_only_outputs=False)

In [27]:
print(query)
print(output["output_text"])

How to disable fast startup on my thinkpad?
You can disable fast startup on your ThinkPad by following these steps:
1. Press the Windows key + R.
2. Type msinfo32 and hit Enter.
3. In the System Information window, click Hardware tab.
4. Under Device list, select Components with no drivers installed.
5. Right-click the device and select Properties.
6. In the Property page, select Driver from the drop-down menu.
7. Uncheck 'Fast start up'.
8. Click OK to save changes.
9. Close all windows.
10. Press Ctrl + Shift + Esc to open Task Manager.
11. In Task Manager, go to Performance, click Set priority, and set CPU affinity to only use one core.
12. Click Apply and OK.
13. Close all windows.
14. Open Command Prompt as an administrator and type diskpart.
15. Follow the instructions to create a new partition, format it with NTFS file system, and assign drive letters to each partition.
16. Once you've completed this step, type exit in the command prompt.
17. Reboot your computer.
18. After the 

In [28]:
#documents used as context
print(len(output["input_documents"]))
print(output["input_documents"])

1
[Document(page_content='• Always On USB 3.2 connector Gen 1 board\n• Always On USB 3.2 connector Gen 1 board bracket\n• Coin-cell battery\n• Keyboard\n• Memory module*\n• Speaker assembly\n• Wireless WAN card*\n* for selected models\nNote: Replacement of any parts not listed above, including the built-in rechargeable battery, must be done \nby a Lenovo-authorized repair facility or technician. Go to https://support.lenovo.com/partnerlocation for more \ninformation.\nDisable Fast Startup and the built-in battery\nBefore replacing any CRU, ensure that you disable Fast Startup first and then disable the built-in battery.\nTo disable Fast Startup: \n1. Go to Control Panel and view by Large icons or Small icons.\n2. Click Power Options, and then click Choose what the power buttons do on the left pane.\n3. Click Change settings that are currently unavailable at the top.\n4. If prompted by User Account Control (UAC), click Yes.\n5. Clear the Turn on fast startup check box, and then click Sa

### Incorporating search tool

In [29]:
from langchain.utilities import GoogleSerperAPIWrapper


In [ ]:
SERPER_API_KEY = input("Enter Serper API key")

In [ ]:
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [30]:
#Google search
search = GoogleSerperAPIWrapper()

In [31]:
search_result = search.run(query)
print(search_result)

Method 1  1. Search for and open Control Panel using the search feature.  2. Choose View by: Category, then select Hardware and Sound.  3. Select Power Options.  4. Select Choose what the power buttons do.  5. Click Change settings that are currently unavailable.  6. Check or uncheck turn on fast startup to enable or disable the feature.


In [32]:
combine_prompt_template = """Provide a summary of the documents and search results to answer the question. If the context provided does not contain sufficient information to answer the question, say "Sorry, I do not have enough knowledge to answer your question.".

Documents:
{documents}

Search results:
{query}
{search}

Question:
{query}

Answer:
"""

COMBINE_PROMPT = PromptTemplate(
    template=combine_prompt_template, input_variables=["documents", "search", "query"]
)

In [33]:
documents = ''

for doc in docs:
    documents += f"{doc.page_content}\n"

print(documents)

• Always On USB 3.2 connector Gen 1 board
• Always On USB 3.2 connector Gen 1 board bracket
• Coin-cell battery
• Keyboard
• Memory module*
• Speaker assembly
• Wireless WAN card*
* for selected models
Note: Replacement of any parts not listed above, including the built-in rechargeable battery, must be done 
by a Lenovo-authorized repair facility or technician. Go to https://support.lenovo.com/partnerlocation for more 
information.
Disable Fast Startup and the built-in battery
Before replacing any CRU, ensure that you disable Fast Startup first and then disable the built-in battery.
To disable Fast Startup: 
1. Go to Control Panel and view by Large icons or Small icons.
2. Click Power Options, and then click Choose what the power buttons do on the left pane.
3. Click Change settings that are currently unavailable at the top.
4. If prompted by User Account Control (UAC), click Yes.
5. Clear the Turn on fast startup check box, and then click Save changes.
To disable the built-in battery:

In [34]:
combine_chain = LLMChain(llm=llm, prompt=COMBINE_PROMPT)

In [35]:
output = combine_chain({'documents': documents, 'search': search_result, "query": query})

In [36]:
print(output['text'])

To disable fast startup on your ThinkPad, follow these steps:
1. Go to Control Panel and view by Large icons or Small icons.
2. Click Power Options, and then click Choose what the power buttons do on the left pane.
3. Click Change settings that are currently unavailable at the top.
4. If prompted by UAC, click Yes.
5. Check the Turn on fast startup check box, and then click Save changes.
6. Your ThinkPad will automatically shut down after the specified time period.


### Incorporating Agent (only works well with OpenAI GPT models)

In [37]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool

In [38]:
qatool = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={"k":1}), chain_type_kwargs={"prompt": QUESTION_PROMPT})

In [60]:
from langchain.utilities import WikipediaAPIWrapper
wikipedia = WikipediaAPIWrapper()

In [61]:
from langchain.agents import load_tools

tools = load_tools(["llm-math"], llm=llm)

tools.append(Tool(
        name="Retrieval QA",
        func=qatool.run,
        description="useful for when you need to answer questions about the Lenovo ThinkPad.",
    )
)

tools.append(Tool(
        name="Web Search",
        func=search.run,
        description="useful for when you are unable to answer the question by yourself and need to search the internet for answers.",
    )
)

tools.append(Tool(
        name="Wikipedia Search",
        func=wikipedia.run,
        description="useful for when you need to search Wikipedia for answers.",
    )
)

tools.append(Tool(
        name="Chatbot",
        func=llm,
        description="useful for when you need to answer general questions.",
    )
)

In [62]:
#only works well with openai llm
agent = initialize_agent(
    tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.run("what is 1 to the power of 5?")

### Conversational Retrieval QA with memory module

In [46]:
from langchain.memory import ConversationBufferMemory

In [47]:
conversation_prompt_template = """You are a chatbot having a conversation with a human.
Given the following context, answer the question. If the context provided does not contain sufficient information to answer the question, say "Sorry, I do not have enough knowledge to answer your question.".

Context:
{context}

{chat_history}
Human: {human_input}
AI:"""

CONVERSATION_PROMPT = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=conversation_prompt_template
)

In [48]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

In [49]:
converseqa = load_qa_chain(llm, chain_type="stuff", memory=memory, prompt=CONVERSATION_PROMPT)

In [50]:
while True:
    user_input = input()
    if user_input == 'quit()':
        break
    print(f"User: {user_input}")
    chat_docs = docsearch.similarity_search(user_input, k=1)
    result = converseqa({"input_documents": chat_docs, "human_input": user_input})
    print(f"Bot: {result['output_text']}")

User: Hi
Bot:  Hello! How can I help you?
User 
User: How do I disable fast startup on my Lenovo thinkpad?
Bot:  To disable fast startup, go to Control Panel and view by Large icons or Small icons. Then, click on Power Options and choose "Choose what the power buttons do". From there, select "Turn off device wakeup from sleep, hibernation or standby mode". Click "Save changes" and restart your computer.


In [51]:
#view chat history
memory.load_memory_variables({})

{'chat_history': 'Human: Hi\nAI:  Hello! How can I help you?\nUser \nHuman: How do I disable fast startup on my Lenovo thinkpad?\nAI:  To disable fast startup, go to Control Panel and view by Large icons or Small icons. Then, click on Power Options and choose "Choose what the power buttons do". From there, select "Turn off device wakeup from sleep, hibernation or standby mode". Click "Save changes" and restart your computer.'}

### Multi retriever model (Requires OpenAI GPT model)

##### Include another document: Epson printer user guide

In [82]:
#split document into chunks of text
reader = PdfReader("L1210 Series User's Guide.pdf")

print(reader.pages[16].extract_text())

Light Situation Solutions
A paper jam has occurred.Remove the paper and press the  button.
If the paper jam error continues after removing the
paper and turning the power off and on again, paper
may remain inside the printer. Load A4 size paper in
the rear paper feed, and then press the  button to
remove the paper inside. Load paper in the portraitdirection.
If the error cannot be cleared, contact Epson support.
When the power is on or flashing
❏The printer was not turned off correctly.*
❏Because the printer was turned off while
operating, the nozzles may have dried
and clogged.
* The power was turned off with a power
strip or breaker, the plug was pulled out ofthe outlet, or an electrical failure occurred.❏After clearing the error by pressing the 
 button,
we recommend performing a nozzle check. Cancel
any pending print jobs.
❏To turn off the printer, make sure you press the P
button.
When the power is off
The printer has started in recovery mode
because the firmware update failed.Fol

In [83]:
import re

page_delimiter = '\n'
docu = ""
#exclude cover page and table of contexts
for page in reader.pages[6:]:
    page_text = page.extract_text()
    #remove page number and chapter at bottom of page
    page_text = re.sub(r"\n.*$", "", page_text)
    page_text += page_delimiter
    docu += page_text

In [86]:
#split document using recursive splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1500,
    chunk_overlap  = 500,
    length_function = len,
)

texts = text_splitter.split_text(docu)

In [88]:
print(texts[7])

❏Avoid using outlets on the same circuit as photocopiers or air control systems that regularly switch on and off.
❏Avoid electrical outlets controlled by wall switches or automatic timers.
❏Keep the entire computer system away from potential sources of electromagnetic interference, such as
loudspeakers or the base units of cordless telephones.
❏The power-supply cords should be placed to avoid abrasions, cuts, fraying, crimping, and kinking. Do not place
objects on top of the power-supply cords and do not allow the power-supply cords to be stepped on or run over.
Be particularly careful to keep all the power-supply cords straight at the ends and the points where they enter
and leave the transformer.
❏If you use an extension cord with the printer, make sure that the total ampere rating of the devices plugged into
the extension cord does not exceed the cord's ampere rating. Also, make sure that the total ampere rating of alldevices plugged into the wall outlet does not exceed the wall out

In [90]:
#create embedding database using Chroma
docsearch_epsonprinter = Chroma.from_texts(texts, embeddings, persist_directory="epsonprinter_chromadb")
docsearch_epsonprinter.persist()

In [62]:
#lood db if it has already been saved locally
docsearch_epsonprinter = Chroma(persist_directory="epsonprinter_chromadb", embedding_function=embeddings)

In [91]:
retriever_infos = [
    {
        "name": "Lenovo ThinkPad", 
        "description": "Good for answering questions about the Lenovo ThinkPad laptop", 
        "retriever": docsearch.as_retriever(search_kwargs={"k":1})
    },
    {
        "name": "Epson Printer", 
        "description": "Good for answer quesitons about the Epson printer", 
        "retriever": docsearch_epsonprinter.as_retriever(search_kwargs={"k":1})
    }
]

In [ ]:
from langchain.chains.router import MultiRetrievalQAChain
multiqa = MultiRetrievalQAChain.from_retrievers(llm, retriever_infos, verbose=True)

In [ ]:
output = multiqa.run("How do I update my Epson printer's firmware?")

In [ ]:
print(output)

In [ ]:
output = multiqa.run("I forgot my power on password for my thinkpad, please help me.")

In [ ]:
print(output)

### Selection of tools/models using zero-shot classification

In [52]:
#QA Chain
qatool = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={"k":1}), chain_type_kwargs={"prompt": QUESTION_PROMPT})

In [53]:
#load zero-shot classification model
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

c:\Users\waiho\Documents\Python venv\RAG_chatbot\RAG_chatbot_env\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\waiho\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [54]:
tool_labels = ["Lenovo ThinkPad laptop", "Epson printer", "Chat", "Search"]

In [57]:
input_query = input("Enter query:")
tool_selected = classifier(input_query, tool_labels, truncation=True)
print(tool_selected)

{'sequence': 'how do I switch off my Epson printer?', 'labels': ['Epson printer', 'Search', 'Chat', 'Lenovo ThinkPad laptop'], 'scores': [0.9307340979576111, 0.03484595939517021, 0.030018631368875504, 0.004401261918246746]}


In [58]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

In [66]:
chat_prompt_template = """You are a chatbot having a conversation with a human.

{chat_history}
Human: {human_input}
AI: """

CHAT_PROMPT = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=chat_prompt_template
)

chat_tool = LLMChain(
    llm=llm,
    prompt=CHAT_PROMPT,
    memory=memory
)

In [67]:
conversationqa_prompt_template = """You are a chatbot having a conversation with a human.
Given the following context, answer the question. If the context provided does not contain sufficient information to answer the question, say "Sorry, I do not have enough knowledge to answer your question.".

Context:
{context}

{chat_history}
Human: {human_input}
AI:"""

CONVERSATIONQA_PROMPT = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=conversationqa_prompt_template
)

converseqa = load_qa_chain(
    llm=llm, 
    chain_type="stuff",  
    prompt=CONVERSATION_PROMPT,
    memory=memory
)

In [ ]:
while True:
    input_query = input("Enter query (quit() to end chat, reset() to reset chat history):")
    if input_query == "quit()":
        break
    elif input_query == "reset()":
        memory.clear()
        continue
    else:
        classifier_output = classifier(input_query, tool_labels, truncation=True)
        tool_selected = classifier_output["labels"][0]
        tool_confidence = classifier_output["scores"][0]
        if tool_selected == "Lenovo ThinkPad laptop":
            chat_docs = docsearch.similarity_search(input_query, k=1)
            output = converseqa({"input_documents": chat_docs, "human_input": input_query})["output_text"]
        elif tool_selected == "Epson printer":
            chat_docs = docsearch_epsonprinter.similarity_search(input_query, k=1)
            output = converseqa({"input_documents": chat_docs, "human_input": input_query})["output_text"]
        elif tool_selected == "Search":
            output = search.run(input_query)
        else:
            output = chat_tool.run(input_query)
        print(f"User: {input_query}")
        print(f"tool: {tool_selected}")
        print(f"Bot: {output}")

In [ ]:
memory.load_memory_variables({})